In [1]:
!pip install simpletransformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 250 kB 4.5 MB/s 
     |████████████████████████████████| 43 kB 2.3 MB/s 
     |████████████████████████████████| 9.2 MB 65.2 MB/s 
     |████████████████████████████████| 7.6 MB 68.2 MB/s 
     |████████████████████████████████| 452 kB 77.3 MB/s 
     |████████████████████████████████| 5.8 MB 64.7 MB/s 
     |████████████████████████████████| 1.3 MB 57.0 MB/s 
     |████████████████████████████████| 1.9 MB 38.2 MB/s 
     |████████████████████████████████| 182 kB 78.9 MB/s 
     |████████████████████████████████| 174 kB 79.9 MB/s 
     |████████████████████████████████| 182 kB 75.3 MB/s 
     |████████████████████████████████| 62 kB 1.8 MB/s 
     |████████████████████████████████| 173 kB 79.3 MB/s 
     |████████████████████████████████| 168 kB 81.7 MB/s 
     |████████████████████████████████| 168 kB 71.1 MB/s 
     |██████████████████████████████

In [2]:
import pandas as pd
data = pd.read_excel('ML_data.xlsx')
data.head()

,word,course,sentence,tag
0,master,course 0,Sentence 0,O
1,machine,course 0,Sentence 0,B-skill
2,learning,course 0,Sentence 0,I-skill
3,on,course 0,Sentence 0,O
4,python,course 0,Sentence 0,B-skill


In [3]:
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [4]:
data["sentence"] = LabelEncoder().fit_transform(data["sentence"])
data.head()

,word,course,sentence,tag
0,master,course 0,0,O
1,machine,course 0,0,B-skill
2,learning,course 0,0,I-skill
3,on,course 0,0,O
4,python,course 0,0,B-skill


In [5]:
data.rename(columns={"sentence":"sentence_id","word":"words","tag":"labels"}, inplace =True)
data["labels"] = data["labels"].str.upper()
data.head()

,words,course,sentence_id,labels
0,master,course 0,0,O
1,machine,course 0,0,B-SKILL
2,learning,course 0,0,I-SKILL
3,on,course 0,0,O
4,python,course 0,0,B-SKILL


In [6]:
X = data[["sentence_id","words"]]
Y = data["labels"]

In [7]:
x_train, x_test, y_train, y_test = train_test_split(X,Y, test_size =0.2)

In [8]:
#building up train data and test data
train_data = pd.DataFrame({"sentence_id":x_train["sentence_id"],"words":x_train["words"],"labels":y_train})
test_data = pd.DataFrame({"sentence_id":x_test["sentence_id"],"words":x_test["words"],"labels":y_test})

In [9]:
train_data

,sentence_id,words,labels
602,158,time,O
932,8,boltzmann,O
576,154,unseen,O
1248,50,aws,O
986,16,lex,O
...,...,...,...
679,167,pacman,O
701,169,cross,O
15,106,make,O
1661,81,data,O


# Model Training

In [10]:
from simpletransformers.ner import NERModel,NERArgs

In [11]:
label = data["labels"].unique().tolist()
label

['O', 'B-SKILL', 'I-SKILL']

In [12]:
args = NERArgs()
args.num_train_epochs = 12
args.learning_rate = 1e-4
args.overwrite_output_dir = True
args.train_batch_size = 32
args.eval_batch_size = 32

In [13]:
model = NERModel('bert', 'bert-base-cased', labels = label, args = args)

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/436M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForTokenClassification: ['cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-cas

Downloading:   0%|          | 0.00/213k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

In [14]:
model.train_model(train_data,eval_data = test_data,acc=accuracy_score)

  0%|          | 0/1 [00:00<?, ?it/s]

Epoch:   0%|          | 0/12 [00:00<?, ?it/s]

Running Epoch 0 of 12:   0%|          | 0/7 [00:00<?, ?it/s]

/usr/local/lib/python3.8/dist-packages/torch/optim/lr_scheduler.py:138: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  warnings.warn("Detected call of `lr_scheduler.step()` before `optimizer.step()`. "


Running Epoch 1 of 12:   0%|          | 0/7 [00:00<?, ?it/s]

...

Running Epoch 8 of 12:   0%|          | 0/7 [00:00<?, ?it/s]

Running Epoch 9 of 12:   0%|          | 0/7 [00:00<?, ?it/s]

Running Epoch 10 of 12:   0%|          | 0/7 [00:00<?, ?it/s]

Running Epoch 11 of 12:   0%|          | 0/7 [00:00<?, ?it/s]

(84, 0.1295177544026436)

In [15]:
result, model_outputs, preds_list = model.eval_model(test_data)
result

  0%|          | 0/1 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/6 [00:00<?, ?it/s]

{'eval_loss': 0.7694160143534342,
 'precision': 0.875,
 'recall': 0.7291666666666666,
 'f1_score': 0.7954545454545454}

In [16]:
prediction, model_output = model.predict(["In this course you will gain practice on machine learning strategies"])
prediction

  0%|          | 0/1 [00:00<?, ?it/s]

Running Prediction:   0%|          | 0/1 [00:00<?, ?it/s]

[[{'In': 'O'},
  {'this': 'O'},
  {'course': 'O'},
  {'you': 'O'},
  {'will': 'O'},
  {'gain': 'O'},
  {'practice': 'O'},
  {'on': 'O'},
  {'machine': 'B-SKILL'},
  {'learning': 'I-SKILL'},
  {'strategies': 'O'}]]